# Using roulette wheel selection, what is the smallest population size and number of generations to find a perfect solution with an individual size of 50?  Give your answer regarding individuals processed: the population size multiplied by the number of generations required.

1. The smallest size of population to find a perfect solution is 50 with 476 generations
1. Total number of individuals (the population size multiplied by the number of generations) is in the range of 10000-25000. This is based on the result table at the end of the notebook

   # GA applied to OneMax

In [1]:
%pip install deap
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Import the DEAP tools and useful libraries (random and matplotlib).

In [2]:
from deap import base
from deap import creator
from deap import tools

import random

import matplotlib.pyplot as plt

Set our Genetic Algorithm parameters

In [3]:
MAX_GENERATIONS = 10000

Set any problem-specific constants here. In this case we need to know how long the string is.

In [4]:
ONE_MAX_LENGTH = 50  # length of bit string to be optimized

Set the random seed. This is important so that we can reproduce runs later on.

In [5]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

Create our toolbox. Note that we can pull in a bunch of predefined operators to tailor our Evolutionary Algorithm, which, of course, in this case is a GA. Notice that it is possible to create our **own** operators and functions to use, which is what we do with our **oneMaxFitness** function below.

In [6]:
toolbox = base.Toolbox()

# create an operator that randomly returns 0 or 1:
toolbox.register("zeroOrOne", random.randint, 0, 1)

# define a single objective, maximizing fitness strategy:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# create the Individual class based on list:
creator.create("Individual", list, fitness=creator.FitnessMax)
#creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

# create the individual operator to fill up an Individual instance:
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, ONE_MAX_LENGTH)

# create the population operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


# fitness calculation:
# compute the number of '1's in the individual
def oneMaxFitness(individual):
    return sum(individual),  # return a tuple


toolbox.register("evaluate", oneMaxFitness)

# genetic operators:

# Tournament selection with tournament size of 3:
toolbox.register("select", tools.selRoulette)

# Single-point crossover:
toolbox.register("mate", tools.cxOnePoint)

# Flip-bit mutation:
# indpb: Independent probability for each attribute to be flipped
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/ONE_MAX_LENGTH)



Here is the main GA loop. We will iterate through it up to the MAX_GENERATIONS parameter and then print out our best individual.

In [7]:
import numpy as np

# for each population_size, grid search over mutation_rate and crossover_rate
result = {}
for populationSize in range(10, 400, 20):

    minGenerationCount = MAX_GENERATIONS + 1
    minGenMutationRate = 0
    minGenCrossoverRate = 0

    for mutationRate in np.arange(0, 0.2, 0.02):
        for crossoverRate in np.arange(0, 1, 0.1):
            population = toolbox.populationCreator(n=populationSize)
            generationCounter = 0

            # calculate fitness tuple for each individual in the population:
            fitnessValues = list(map(toolbox.evaluate, population))
            for individual, fitnessValue in zip(population, fitnessValues):
                individual.fitness.values = fitnessValue

            # extract fitness values from all individuals in population:
            fitnessValues = [individual.fitness.values[0] for individual in population]

            # initialize statistics accumulators:
            maxFitnessValues = []
            meanFitnessValues = []

            # main evolutionary loop:
            # stop if max fitness value reached the known max value
            # OR if number of generations exceeded the preset value:
            while True:
                if generationCounter > MAX_GENERATIONS:
                    # print(f"Stopping since max generations reached. Population size: {populationSize}, Mutation rate: {mutationRate}, Crossover rate: {crossoverRate}")
                    break

                # update counter:
                generationCounter = generationCounter + 1

                # apply the selection operator, to select the next generation's individuals:
                offspring = toolbox.select(population, len(population))
                # clone the selected individuals:
                offspring = list(map(toolbox.clone, offspring))

                # apply the crossover operator to pairs of offspring:
                for child1, child2 in zip(offspring[::2], offspring[1::2]):
                    if random.random() < crossoverRate:
                        toolbox.mate(child1, child2)
                        del child1.fitness.values
                        del child2.fitness.values

                for mutant in offspring:
                    if random.random() < mutationRate:
                        toolbox.mutate(mutant)
                        del mutant.fitness.values

                # calculate fitness for the individuals with no previous calculated fitness value:
                freshIndividuals = [ind for ind in offspring if not ind.fitness.valid]
                freshFitnessValues = list(map(toolbox.evaluate, freshIndividuals))
                for individual, fitnessValue in zip(freshIndividuals, freshFitnessValues):
                    individual.fitness.values = fitnessValue

                # replace the current population with the offspring:
                population[:] = offspring

                # collect fitnessValues into a list, update statistics and print:
                fitnessValues = [ind.fitness.values[0] for ind in population]

                maxFitness = max(fitnessValues)
                meanFitness = sum(fitnessValues) / len(population)
                maxFitnessValues.append(maxFitness)
                meanFitnessValues.append(meanFitness)

                if maxFitness >= ONE_MAX_LENGTH:
                    if generationCounter < minGenerationCount:
                        minGenerationCount = generationCounter
                        minGenMutationRate = mutationRate
                        minGenCrossoverRate = crossoverRate
                    # print(f"Stopping since max fitness reached. Population size: {populationSize}, Mutation rate: {mutationRate}, Crossover rate: {crossoverRate}")
                    break

    if minGenerationCount <= MAX_GENERATIONS:
        print(f"{populationSize*minGenerationCount} Population size: {populationSize}, Min generations: {minGenerationCount}, Min mutation rate: {minGenMutationRate}, Min crossover rate: {minGenCrossoverRate}")
        result[populationSize] = (
            populationSize * minGenerationCount,
            minGenerationCount,
            minGenMutationRate,
            minGenCrossoverRate,
        )
    else:
        print(f"Population size: {populationSize}, No solution found")

Population size: 10, No solution found


Population size: 30, No solution found


23800 Population size: 50, Min generations: 476, Min mutation rate: 0.08, Min crossover rate: 0.9


20300 Population size: 70, Min generations: 290, Min mutation rate: 0.14, Min crossover rate: 0.7000000000000001


19170 Population size: 90, Min generations: 213, Min mutation rate: 0.06, Min crossover rate: 0.9


...

18910 Population size: 310, Min generations: 61, Min mutation rate: 0.0, Min crossover rate: 0.9


24420 Population size: 330, Min generations: 74, Min mutation rate: 0.02, Min crossover rate: 0.9


23100 Population size: 350, Min generations: 66, Min mutation rate: 0.06, Min crossover rate: 0.7000000000000001


23680 Population size: 370, Min generations: 64, Min mutation rate: 0.1, Min crossover rate: 0.8


19890 Population size: 390, Min generations: 51, Min mutation rate: 0.1, Min crossover rate: 0.9


In [8]:
import pandas as pd

df = pd.DataFrame(columns=["Population size", "Population size * Generation", "Generations", "Mutation rate", "Crossover rate"])
for key, value in result.items():
    df.loc[df.__len__()] = {
        "Population size": key,
        "Population size * Generation": value[0],
        "Generations": value[1],
        "Mutation rate": value[2],
        "Crossover rate": value[3],
    }
df

,Population size,Population size * Generation,Generations,Mutation rate,Crossover rate
0,50,23800,476,0.08,0.9
1,70,20300,290,0.14,0.7
2,90,19170,213,0.06,0.9
3,110,10780,98,0.14,0.9
4,130,10010,77,0.02,0.8
5,150,11700,78,0.06,0.7
6,170,20400,120,0.12,0.9
7,190,13300,70,0.02,0.9
8,210,15750,75,0.02,0.9
9,230,16100,70,0.06,0.8
